In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
from pytenet.operation_thc import apply_thc_mpo_and_compress, add_mps_and_compress
from pytenet.operation import vdot, add_mps
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import scipy.sparse.linalg as spla
from pytenet.krylov_time_evo import ED_time_evo, Krylov_evo_using_vecs_single_step, Krylov_time_evo_using_vecs, Krylov_evo_using_built_space, create_Krylov_space, gram_schmidt

In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [3]:
#ref Hamiltonian as matrix
H_ref = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_thc = scipy.io.mmread('../data_water/H_water_thc.mtx').tocsr()

#initial state
filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
with open(filename, 'rb') as file:
    initial_state = pickle.load(file)

#thc mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)
r_THC = int((len(H_mu_nu_list_spin_layer)-1)**(1/2) / 2)
    

In [28]:
N_krylov = 4

psi_krylov_ref = Krylov_time_evo_using_vecs(H_ref, N_krylov, initial_state.as_vector(), 1, 0.05)
psi_ed = ED_time_evo(H_ref, initial_state.as_vector(), 0.05)
print('total swift:', norm(psi_ed - initial_state.as_vector()))
print(norm(psi_krylov_ref - psi_ed))

[[-4.72803783e-01-8.80997071e-01j]
 [-1.46584636e-02+8.99739033e-03j]
 [ 1.76683447e-03+1.27867724e-03j]
 [ 1.13656067e-04-3.86514204e-04j]]
total swift: 1.7162772406158837
4.078012059353811e-05


In [5]:
'''
for i in range(n):
first update foldername
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)
then calculate time-evolved state (with re-ortho)
'''
#不需要创造很多文件夹储存文件，对每个文件好好命名然后都存在同一个文件夹即可

'\nfor i in range(n):\nfirst update foldername\ngenerate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)\nthen calculate time-evolved state (with re-ortho)\n'

In [6]:
Krylov_vector_list = []

Krylov_vector_list.append(initial_state.as_vector())

for i in range (N_krylov-1):
    krylov_vector = H_ref@ Krylov_vector_list[i]
    krylov_vector /= norm(krylov_vector)
        
    for j in range (i+1):    
        krylov_vector +=  -np.vdot(Krylov_vector_list[j], krylov_vector)*Krylov_vector_list[j]
    
    krylov_vector /= norm(krylov_vector)
    Krylov_vector_list.append(krylov_vector)

In [7]:
#space_test = create_Krylov_space(4, H_mu_nu_list_spin_layer, copy.deepcopy(initial_state), 1e-10, 100, r_THC)

In [20]:
trunc_tol = 0
max_bond = 35

Krylov_space = []
    
v0 = copy.deepcopy(initial_state)
v0.orthonormalize('left')
v0.orthonormalize('right')
Krylov_space.append(v0)

print(v0.bond_dims)
print(v0.as_vector() - initial_state.as_vector())



v1 = apply_thc_mpo_and_compress(H_mu_nu_list_spin_layer, copy.deepcopy(v0), trunc_tol, 2*max_bond, r_THC)
print(v1.as_vector() - H_ref@v0.as_vector())
#v1.orthonormalize('left')
#v1.orthonormalize('right')
temp = copy.deepcopy(v0)
temp.A[0] = -vdot(v1, temp)* temp.A[0]
v1 =  add_mps_and_compress(copy.deepcopy(v1), temp, trunc_tol, max_bond)
v1.orthonormalize('left')
v1.orthonormalize('right')
Krylov_space.append(v1)
print(v1.bond_dims)

#print(v1.as_vector() - H_ref@v0.as_vector())

print(norm(v1.as_vector() - Krylov_vector_list[1]))


[1, 4, 16, 31, 31, 16, 4, 1]
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
[1, 4, 16, 35, 35, 16, 4, 1]
0.00035098443767252435


In [21]:
for i in range(2, N_krylov, 1):
    
    v_i = apply_thc_mpo_and_compress(H_mu_nu_list_spin_layer, copy.deepcopy(Krylov_space[i-1]), trunc_tol, 2*max_bond, r_THC)
    v_i.orthonormalize('left')
    v_i.orthonormalize('right')
    
    temp1 = copy.deepcopy(Krylov_space[i-2])
    temp1.A[0] = -vdot(v_i, temp1)* temp1.A[0]
    v_i =  add_mps(copy.deepcopy(v_i), temp1)
    #v_i.orthonormalize('left')
    #v_i.orthonormalize('right')
    
    temp2 = copy.deepcopy(Krylov_space[i-1])
    temp2.A[0] = -vdot(v_i, temp2)* temp2.A[0]
    v_i =  add_mps_and_compress(copy.deepcopy(v_i), temp2, trunc_tol, max_bond)
    v_i.orthonormalize('left')
    v_i.orthonormalize('right')
    print(v_i.bond_dims)
    Krylov_space.append(v_i)
        
        # print(vdot(Krylov_space[i], Krylov_space[i-1]))
        # print(vdot(Krylov_space[i], Krylov_space[i-2]))

[1, 4, 16, 35, 35, 16, 4, 1]
[1, 4, 16, 35, 35, 16, 4, 1]


In [22]:
# space_test_store = copy.deepcopy(space_test)

In [23]:
# space_test = copy.deepcopy(space_test)
# space_test = space_test[:4]

In [24]:
# space_test_vec = [krylov_mps.as_vector() for krylov_mps in space_test]
# space_test_vec = gram_schmidt(space_test_vec)

In [25]:
space_test_vec = []
for i in range (len(Krylov_space)):
    temp = Krylov_space[i].as_vector()
    temp /= norm(temp)
    space_test_vec.append(temp)

space_test_vec = gram_schmidt(space_test_vec)

In [29]:
#H_thc
vec_test = Krylov_evo_using_built_space(H_ref, space_test_vec, 0.05)
print(norm(vec_test - psi_krylov_ref))
print(norm(vec_test - psi_ed))

5.6108776621505995e-06
4.114603732732838e-05


In [ ]:
# 40:
# 7.159356839258043e-13
# 5.486690011281797e-06

In [ ]:
# print(norm(vec_test - psi_ed))

In [ ]:
# for i in range (len(space_test_vec)):
#     for j in range (len(space_test_vec)):
#         if i != j:
#             print(i, j)
#             print(np.vdot(space_test_vec[i], space_test_vec[j])) 

In [ ]:
dt = 0.03

TN = np.zeros([len(space_test_vec),len(space_test_vec)])
for i in range (TN.shape[0]):
    for j in range (TN.shape[1]):
        #if abs(i - j) < 2:
        TN[i, j] = np.vdot(space_test_vec[i], H_ref @space_test_vec[j])
                
c1 = np.zeros([len(space_test_vec), 1])
c1[0,0] = 1
exp_TN = spla.expm(-1j*dt*TN)
c_reduced = exp_TN@ c1

print(c_reduced)


[[-8.19779916e-01+5.72583803e-01j]
 [ 6.25416544e-03+8.29909387e-03j]
 [ 4.97250480e-04-6.54384024e-04j]
 [-1.01664789e-04-4.66367026e-05j]]


/tmp/ipykernel_3010123/2780000454.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  TN[i, j] = np.vdot(space_test_vec[i], H_ref @space_test_vec[j])


In [ ]:
psi_evloved = np.zeros_like(space_test_vec[0], dtype=np.complex128)

for i in range (len(space_test_vec)):
    psi_evloved += c_reduced[i] * space_test_vec[i]

In [ ]:
print(norm(psi_evloved - psi_ed))

0.0002521637526303062


In [ ]:
print(norm(psi_krylov_ref - psi_ed))


5.4866900147348975e-06


In [ ]:
print(norm(psi_evloved - psi_krylov_ref))

0.00025208484466916634
